In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense,CuDNNLSTM, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from janome.tokenizer import Tokenizer
import numpy as np
import random
import glob
import sys
import io
import re
import os

t = Tokenizer()

Using TensorFlow backend.


In [2]:
char_indices = {}  # 辞書初期化
indices_char = {}  # 逆引き辞書初期化
texts= []
chars = []
parts = []
paths = glob.glob('C:/Users/hata/sozoenshu/small_data/*.txt')
for path in paths:
    word = []
    part = []
    print('reading ' , path)
    binarydata = open(path,'rb').read()
    text = binarydata.decode('shift_jis')
    text = re.split(r'\-{5,}',text)[2]
    text = re.split(r'底本：',text)[0]
    text = text.replace('|','')
    text = text.replace('\u3000','')
    text = re.sub(r'《.+?》','',text) # ルビをとる
    text = re.sub(r'※','',text) # ※をとる
    text = re.sub(r'sentimentalisme','センチメンタリズム',text)
    text = re.sub(r'\r\n','',text)
    text = re.sub(r'―','',text)
    text = re.sub(r'［＃.+?］','',text) # 入力注をとる
    texts.append(text)
    print('corpus length:', len(text))
    malist = t.tokenize(text)
    for w in malist:
        word.append(w.surface)
        part.append(w.part_of_speech.split(','))
    chars.append(word)
    parts.append(part)

POS = []
for i in range(len(parts)):
    for j in range(len(parts[i])):
        if parts[i][j] not in POS:
            POS.append(parts[i][j])

print('Part of Speech:',POS)
print('Part of Speech length:', len(POS))
print('texts length:',len(texts))
print('chars length:', len(chars))

# 辞書の作成
global_chars = []
for i in range(len(chars)):
    print('chars['+str(i)+']:', len(chars[i]))
    for word in chars[i]:
        if not word in global_chars:  # 未登録なら
            global_chars.append(word)  # 登録する

char_indices = dict((c,i) for i,c in enumerate(global_chars))
indices_char = dict((i,c) for i,c in enumerate(global_chars))

POS_indices = dict((str(c),i) for i,c in enumerate(POS))
indices_POS = dict((i,str(c)) for i,c in enumerate(POS))

print('global_chars length', len(global_chars))

del text
del texts
del paths

reading  C:/Users/hata/sozoenshu/small_data\susanono_mikoto.txt
corpus length: 42370
Part of Speech: [['名詞', '数', '*', '*'], ['名詞', '一般', '*', '*'], ['助詞', '連体化', '*', '*'], ['助詞', '係助詞', '*', '*'], ['助詞', '格助詞', '一般', '*'], ['動詞', '自立', '*', '*'], ['助動詞', '*', '*', '*'], ['記号', '句点', '*', '*'], ['名詞', '副詞可能', '*', '*'], ['助詞', '接続助詞', '*', '*'], ['記号', '読点', '*', '*'], ['動詞', '非自立', '*', '*'], ['形容詞', '自立', '*', '*'], ['名詞', '接尾', '助数詞', '*'], ['名詞', '形容動詞語幹', '*', '*'], ['連体詞', '*', '*', '*'], ['接頭詞', '名詞接続', '*', '*'], ['副詞', '一般', '*', '*'], ['名詞', '非自立', '助動詞語幹', '*'], ['名詞', '固有名詞', '地域', '一般'], ['名詞', '接尾', '一般', '*'], ['助詞', '並立助詞', '*', '*'], ['名詞', '非自立', '副詞可能', '*'], ['副詞', '助詞類接続', '*', '*'], ['名詞', '固有名詞', '人名', '名'], ['名詞', '代名詞', '一般', '*'], ['動詞', '接尾', '*', '*'], ['助詞', '副詞化', '*', '*'], ['接続詞', '*', '*', '*'], ['名詞', '固有名詞', '地域', '国'], ['助詞', '副助詞', '*', '*'], ['名詞', '非自立', '一般', '*'], ['助詞', '副助詞／並立助詞／終助詞', '*', '*'], ['名詞', 'サ変接続', '*', '*'], ['助詞', '格助詞', '連語', '

In [3]:
# 学習データを作る
maxlen = 12
step = 8
X = []
Y = []
sentences = []
next_chars = []
pos = []
next_pos = []
for j in range(len(chars)):
    for i in range(0, len(chars[j]) - maxlen, step):
        sentences.append(chars[j][i: i + maxlen])
        next_chars.append(chars[j][i + maxlen])
        pos.append(parts[j][i: i + maxlen])
        next_pos.append(parts[j][i + maxlen])
    print('nb sequences:', len(sentences))
    print('Vectorization...')
    x = np.zeros((len(sentences), maxlen, len(global_chars)+len(POS)), dtype=np.bool)
    y = np.zeros((len(sentences),len(global_chars)+len(POS)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence): #sentence = sentences[i], char = sentence[t]
            x[i, t, char_indices[char]] = 1
            x[i, t, len(global_chars)+POS_indices[str(pos[i][t])]] = 1
        y[i, char_indices[next_chars[i]]] = 1
        y[i, len(global_chars)+POS_indices[str(pos[i][t])]] = 1
        #print(x)
        #print(y)
    X.append(x)
    Y.append(y)

nb sequences: 3649
Vectorization...


In [4]:
# build the model: a single LSTM
def loss(y, pred):
    s = pred-y
    s = np.sum(s)
    return (abs(s)*10)**2

print('Build model...')
learning_rate = 0.01
model = Sequential()

model.add(CuDNNLSTM(256, input_shape=(maxlen, len(global_chars)+len(POS))))
model.add(BatchNormalization())
model.add(Dropout(0.15))

model.add(Dense(512, activation='relu'))

model.add(Dense((len(global_chars)+len(POS)), activation='softmax'))

print(model.summary())

optimizer = 'Adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer) 

Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 256)               3854336   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 3506)              1798578   
Total params: 5,785,522
Trainable params: 5,785,010
Non-trainable params

In [5]:
def sample(preds, temperature=1.0):
    preds = preds[:len(global_chars)]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    #char_preds = preds[:len(global_chars)]
    #pos_preds = preds[len(global_chars):]
    #return np.argmax(char_preds), np.argmax(pos_preds)
    return np.argmax(probas)

for epochs in range(100):
    for number_of_dataset in range(len(X)):
        for iteration in range(1,3):
            print('\n')
            print('-' *50)
            print('epochs: ', epochs+1)
            print('number_of_dataset: ', number_of_dataset)
            print('繰り返し回数: ', iteration)
            model.fit(X[number_of_dataset], Y[number_of_dataset], batch_size=2048, shuffle = True, epochs=1)

            #start_index = random.randint(0, len(chars[number_of_dataset])-maxlen-1)
            start_index = 0
            
            for diversity in [1.2]:
                print()
                print('-----diveristy', diversity)

                generated = ''
                sentence_chars = chars[number_of_dataset][start_index: start_index + maxlen]
                sentence_parts = parts[number_of_dataset][start_index: start_index + maxlen]
                sentence = ''.join(sentence_chars)
                generated += sentence
                print('----- Seedを生成しました: "' + sentence + '"')
                sys.stdout.write(generated)

                for i in range(10):
                    x = np.zeros((len(sentences), maxlen, len(global_chars)+len(POS)), dtype=np.bool)
                    for t, char in enumerate(sentence_chars):
                        x[0, t, char_indices[char]] = 1.
                        x[0, t, POS_indices[str(sentence_parts[t])]] = 1.

                    preds = model.predict(x, verbose=0)[0]
                    next_index = sample(preds, diversity)
                    next_char = indices_char[next_index]
                    generated += next_char
                    sentence_chars = sentence_chars[1:]
                    sentence_chars.append(next_char)
                    
                    sys.stdout.write(next_char)
                    sys.stdout.flush()



--------------------------------------------------
epochs:  1
number_of_dataset:  0
繰り返し回数:  1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
3649/3649 [==============================] - 6s 2ms/step - loss: 16.2054

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は打心もち勝っ組重苦しい飛び起きる圧さ凱歌感じ呑める

--------------------------------------------------
epochs:  1
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 261us/step - loss: 15.5464

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は笠誰垂れ日暮真底無くやす自失蛇深い

--------------------------------------------------
epochs:  2
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 266us/step - loss: 14.7420

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は妬まし青々しい疑う仄明）輝き救い出すただ今漏らし

--------------------------------------------------
epochs:  2
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [========

3649/3649 [==============================] - 1s 263us/step - loss: 2.5319

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はと、ののの、とし、と

--------------------------------------------------
epochs:  16
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 264us/step - loss: 2.3858

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は、のもで、のも、静と

--------------------------------------------------
epochs:  16
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 259us/step - loss: 2.2507

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は、しも、もものののの

--------------------------------------------------
epochs:  17
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 263us/step - loss: 2.1258

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はものだ、の、しとのも

--------------------------------------------------

3649/3649 [==============================] - 1s 273us/step - loss: 1.5129

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は向うとににこの向うこの川筋のな

--------------------------------------------------
epochs:  31
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 270us/step - loss: 1.5139

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は向うになになが人煙な春と

--------------------------------------------------
epochs:  32
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 263us/step - loss: 1.5127

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はそうなにな彼にのななくな

--------------------------------------------------
epochs:  32
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 264us/step - loss: 1.5124

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は向うがな高天原ににになと若者

-------------------------------------

3649/3649 [==============================] - 1s 262us/step - loss: 1.4831

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はににありががただ向うに若者が

--------------------------------------------------
epochs:  47
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 253us/step - loss: 1.4820

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はがなかっただ彼がが。向うにに

--------------------------------------------------
epochs:  47
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 263us/step - loss: 1.4833

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はが彼ただなよがん若者が。

--------------------------------------------------
epochs:  48
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 263us/step - loss: 1.4809

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はただがないな事情かた。が」

-------------------------------------

3649/3649 [==============================] - 1s 266us/step - loss: 1.4685

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はこのこの若者かこのに若者をにに

--------------------------------------------------
epochs:  62
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 267us/step - loss: 1.4718

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はと云うになさただにもない事

--------------------------------------------------
epochs:  63
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 263us/step - loss: 1.4675

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は彼ががに人がががなに

--------------------------------------------------
epochs:  63
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 265us/step - loss: 1.4682

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は木。ん彼がしかし部落にに彼

---------------------------------------

3649/3649 [==============================] - 1s 266us/step - loss: 1.4588

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は向うがなに彼になの事情。

--------------------------------------------------
epochs:  78
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 260us/step - loss: 1.4563

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は空かてな所以前ななくらいな

--------------------------------------------------
epochs:  78
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 252us/step - loss: 1.4609

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は素くらいな言葉を所女そうとな

--------------------------------------------------
epochs:  79
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 250us/step - loss: 1.4552

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はこの若者ににの若者なが、彼

--------------------------------------

3649/3649 [==============================] - 1s 262us/step - loss: 1.4510

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はにと後にをと申す事を事

--------------------------------------------------
epochs:  93
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 271us/step - loss: 1.4516

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はが素かな事にも」三事

--------------------------------------------------
epochs:  94
number_of_dataset:  0
繰り返し回数:  1
Epoch 1/1
3649/3649 [==============================] - 1s 267us/step - loss: 1.4521

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今は彼がかにに手足をに彼の

--------------------------------------------------
epochs:  94
number_of_dataset:  0
繰り返し回数:  2
Epoch 1/1
3649/3649 [==============================] - 1s 282us/step - loss: 1.4496

-----diveristy 1.2
----- Seedを生成しました: "一高天原の国も春になった。今は"
一高天原の国も春になった。今はが彼かの向うとにかに苦しく

---------------------------------------------

In [6]:
a=['a','b','c']
a=a[1:]
a.append('de')
print(a)

['b', 'c', 'de']


In [ ]:
text = s
text =Tokenizer().tokenize(text, wakati=True)  # 分かち書きする
chars = text
matasaburo = text

for word in chars:
    if not word in char_indices:  # 未登録なら
        char_indices[word] = count  # 登録する      
        count +=1
        print(count,word)  # 登録した単語を表示
        
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 8
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x0 = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y0 = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x0[i, t, char_indices[char]] = 1
    y0[i, char_indices[next_chars[i]]] = 1

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)    

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2]:  # diversity 
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        # sentence はリストなので文字列へ変換して使用
        generated += "".join(sentence)
        print(sentence)
        
        # sentence はリストなので文字列へ変換して使用
        print('----- Generating with seed: "' + "".join(sentence)+ '"')
        sys.stdout.write(generated)


        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:]
            # sentence はリストなので append で結合する
            sentence.append(next_char)  

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])